    These are the libraries I'm using

In [1]:
import pandas as pd
import json
import plotly.plotly as py
import plotly.graph_objs as go

pd.__version__

u'0.20.1'

    Reading in the data

In [15]:
df = pd.read_csv("City Populations.csv")

with open("IL Cities.geojson") as f:
    cities = json.load(f)

    Creating a dictionary of Key = City Name; Value = Geographical Data
    This will print out the number of cities that do not translate.

In [22]:
geo_dict = {}
not_in = 0

for i in range(len(cities['features'])):
    name = cities['features'][i]['properties']['name'][:-4]
    if name in df["City"].unique():
        geo_dict[name] = cities['features'][i]
    else:
        not_in += 1
print "Not in:", not_in

Not in: 389


In [4]:
ser = pd.Series(geo_dict.values(), index = geo_dict.keys())
ser.name = "Location"

In [5]:
df = df.join(ser, on="City")
df = df.drop(df.index[0])

    Adding the color scheme

In [6]:
colors = ['#ffffe0','#fffddb','#fffad7','#fff7d1','#fff5cd','#fff2c8',
          '#fff0c4','#ffedbf','#ffebba','#ffe9b7','#ffe5b2','#ffe3af',
          '#ffe0ab','#ffdda7','#ffdba4','#ffd9a0','#ffd69c','#ffd399',
          '#ffd196','#ffcd93','#ffca90','#ffc88d','#ffc58a','#ffc288',
          '#ffbf86','#ffbd83','#ffb981','#ffb67f','#ffb47d','#ffb17b',
          '#ffad79','#ffaa77','#ffa775','#ffa474','#ffa172','#ff9e70',
          '#ff9b6f','#ff986e','#ff956c','#fe916b','#fe8f6a','#fd8b69',
          '#fc8868','#fb8567','#fa8266','#f98065','#f87d64','#f77a63',
          '#f67862','#f57562','#f37261','#f37060','#f16c5f','#f0695e',
          '#ee665d','#ed645c','#ec615b','#ea5e5b','#e85b59','#e75859',
          '#e55658','#e45356','#e35056','#e14d54','#df4a53','#dd4852',
          '#db4551','#d9434f','#d8404e','#d53d4d','#d43b4b','#d2384a',
          '#cf3548','#cd3346','#cc3045','#ca2e43','#c72b42','#c52940',
          '#c2263d','#c0233c','#be213a','#bb1e37','#ba1c35','#b71933',
          '#b41731','#b2152e','#b0122c','#ac1029','#aa0e27','#a70b24',
          '#a40921','#a2071f','#a0051c','#9d0419','#990215','#970212',
          '#94010e','#91000a','#8e0006','#8b0000', '#8b0000']

scl = dict(zip(range(0, 101), colors))

{0: '#ffffe0', 1: '#fffddb', 2: '#fffad7', 3: '#fff7d1', 4: '#fff5cd', 5: '#fff2c8', 6: '#fff0c4', 7: '#ffedbf', 8: '#ffebba', 9: '#ffe9b7', 10: '#ffe5b2', 11: '#ffe3af', 12: '#ffe0ab', 13: '#ffdda7', 14: '#ffdba4', 15: '#ffd9a0', 16: '#ffd69c', 17: '#ffd399', 18: '#ffd196', 19: '#ffcd93', 20: '#ffca90', 21: '#ffc88d', 22: '#ffc58a', 23: '#ffc288', 24: '#ffbf86', 25: '#ffbd83', 26: '#ffb981', 27: '#ffb67f', 28: '#ffb47d', 29: '#ffb17b', 30: '#ffad79', 31: '#ffaa77', 32: '#ffa775', 33: '#ffa474', 34: '#ffa172', 35: '#ff9e70', 36: '#ff9b6f', 37: '#ff986e', 38: '#ff956c', 39: '#fe916b', 40: '#fe8f6a', 41: '#fd8b69', 42: '#fc8868', 43: '#fb8567', 44: '#fa8266', 45: '#f98065', 46: '#f87d64', 47: '#f77a63', 48: '#f67862', 49: '#f57562', 50: '#f37261', 51: '#f37060', 52: '#f16c5f', 53: '#f0695e', 54: '#ee665d', 55: '#ed645c', 56: '#ec615b', 57: '#ea5e5b', 58: '#e85b59', 59: '#e75859', 60: '#e55658', 61: '#e45356', 62: '#e35056', 63: '#e14d54', 64: '#df4a53', 65: '#dd4852', 66: '#db4551', 67: 

    Adding the color "value" to the dataframe

In [7]:
def get_scl(obj):
    frac = obj / 10000
    return scl[frac]

df['Color'] = df['Population'].apply(get_scl)

                                      City  Population  \
1                                   Aurora      200661   
2                                 Rockford      148278   
3                                   Joliet      147861   
4                               Naperville      147100   
5                              Springfield      116565   
6                                   Peoria      115070   
7                                    Elgin      112111   
8                                 Waukegan       88475   
9                                Champaign       86096   
10                                  Cicero       83886   
11                             Bloomington       78292   
12                       Arlington Heights       75926   
13                                Evanston       75527   
14                              Schaumburg       74693   
15                             Bolingbrook       74306   
16                                 Decatur       73254   
17            

    This seems to be the issue. Data is in layers?

In [8]:
layers_ls = []
for i in df.index:
    item_dict = dict(sourcetype = 'geojson',
                     source = df.loc[i]['Location'],
                     type = 'fill',
                     color = df.loc[i]['Color'])
    layers_ls.append(item_dict)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [9]:
mapbox_access_token = "pk.eyJ1Ijoiam9uZXNqcDYiLCJhIjoiY2ozcWc1aTY2MDFlZDMzbnVpa3hiN2I2ZSJ9.CdLfET8OqcoZkCUVVeplwg"

In [10]:
colorscl = [[i * .01, v] for i,v in enumerate(colors)]

In [11]:
data = go.Data([
            go.Scattermapbox(
                    lat = [0],
                    lon = [0],
                    marker = go.Marker(
                                  cmax=101,
                                  cmin=0,
                                  colorscale = colorscl,
                                  showscale = True,
                                  autocolorscale=False,
                                  color=range(0,101),
                                  colorbar= go.ColorBar(
                                                 len = .89
                                                        )
                                       ),
                    mode = 'markers')
                     ])

In [12]:
layout = go.Layout(
    title = 'City Populations',
    height=1050,
    width=800,
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        layers= layers_ls,
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=39.03,
            lon=-105.7
        ),
        pitch=0,
        zoom=5.5,
        style='light'
    ),
)

In [13]:
fig = dict(data = data, layout=layout)
py.image.save_as(fig, filename='image/test.jpeg',
                 width = 750, height= 575)
from IPython.display import Image
Image('image/test.jpeg')

py.image.ishow("image/test.jpeg")

PlotlyRequestError: 
<html><head>
<meta http-equiv="content-type" content="text/html;charset=utf-8">
<title>502 Server Error</title>
</head>
<body text=#000000 bgcolor=#ffffff>
<h1>Error: Server Error</h1>
<h2>The server encountered a temporary error and could not complete your request.<p>Please try again in 30 seconds.</h2>
<h2></h2>
</body></html>
